In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1 PREPARE THE DATA


In [3]:
t1 = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
t2 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv')
t3 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
train_dataset=pd.concat([t1,t2,t3],ignore_index=True)
train_dataset.shape


(2078, 4)

In [4]:
t1.generated.value_counts()

generated
0    1375
1       3
Name: count, dtype: int64

In [5]:
train_dataset.to_csv('/kaggle/working/data.csv')
train_dataset.generated.value_counts()

generated
0    1375
1     703
Name: count, dtype: int64

However，there are some problems with the text, so we rectify them and get the improved data-set, finetune.csv.

In [6]:
train_dataset=pd.read_csv("/kaggle/input/finetuned-data/finetuned.csv")
trainset,validset=train_test_split(train_dataset, test_size=0.2, random_state=42)
trainset.to_csv('/kaggle/working/trainset.csv')
validset.to_csv('/kaggle/working/validset.csv')
print(trainset.shape,validset.shape)

(1662, 7) (416, 7)


# 2 NERUAL NETWORK CONSTRUCTION

In [ ]:
# Define paths to BERT preprocessor and encoder
preprocessor_path = "/kaggle/input/bertpreprocess/bertpreprocess"
encoder_path = "/kaggle/input/encoderbert/encoder"

# Load BERT preprocessor and encoder as KerasLayers
bert_preprocessor = hub.KerasLayer(preprocessor_path)
bert_encoder = hub.KerasLayer(encoder_path, trainable=True)

# Define input layer
input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text_input")

# Apply BERT preprocessor and encoder to the input
preprocessed_inputs = bert_preprocessor(input_layer)
encoded_outputs = bert_encoder(preprocessed_inputs)

# Apply dropout to the pooled output
dropout_rate = 0.2
pooled_output_dropout = tf.keras.layers.Dropout(dropout_rate)(encoded_outputs["pooled_output"])

# Output layer for binary classification
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(pooled_output_dropout)

# Create the model using Functional API
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [ ]:
def load_and_process_data(train_path, valid_path, test_path):
    trainset = pd.read_csv(train_path)
    validset = pd.read_csv(valid_path)
    test_data = pd.read_csv(test_path)

    X_train_text, X_val_text, X_test_text = (
        trainset["text"],
        validset["text"],
        test_data["text"],
    )
    y_train_labels, y_val_labels = trainset["generated"], validset["generated"]

    X_train, X_val, y_train, y_val = (
        np.array(X_train_text),
        np.array(X_val_text),
        np.array(y_train_labels),
        np.array(y_val_labels),
    )

    return X_train, X_val, y_train, y_val, X_test_text, test_data["id"]

# Load and preprocess data
train_path = "/kaggle/working/trainset.csv"
valid_path = "/kaggle/working/validset.csv"
test_path = "/kaggle/input/llm-detect-ai-generated-text/test_essays.csv"

X_train, X_val, y_train, y_val, X_test_text, test_ids = load_and_process_data(
    train_path, valid_path, test_path
)
# Train the model
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
)

history = model.fit(
    X_train,
    y_train,
    epochs=30,
    validation_split=0.2,  # 20% of training data will be used as validation set
    callbacks=[early_stopping_callback],
)

# Make predictions on the test set
y_test_predictions = model.predict(X_test_text).flatten()

# Concatenate ID and predictions and save to CSV
submit = pd.DataFrame({"id": test_ids, "generated": y_test_predictions})
submit.to_csv("submission.csv", index=False)